Just a simple jupyter to study stuff (embedding and attention)

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from DatasetCreator.DatasetCreator import DatasetCreator
from sympy import sin, Symbol, log, exp 
from eq_learner_architectures import convconv_PoseNet

ModuleNotFoundError: No module named 'DatasetCreator'

In [ ]:
class Network():
    def __init__(self):
        layer = torch.nn.Embedding(3,2)
        final_layer = nn.Linear(2,1)
        activation_fun = nn.Sigmoid(final)
    def __call__(inp):
        partial = layer(sol)
        final = nn.Sigmoid(final_layer(partial))
        return final

In [ ]:
#Encoder Point Cloud data
x = Symbol('x')
basis_functions = [x,sin,log,exp]
support = np.arange(-2,2, 0.01)

fun_generator = DatasetCreator(basis_functions, max_linear_terms=2,max_binomial_terms=3,max_N_terms=0, max_compositions=2, division_on=False)
input_network, output_network =  fun_generator.generate_batch(support,20)
input_network = np.nan_to_num(input_network)
first_encoder = convconv_PoseNet.PoseNetEncoder(len(support),2)
input_tensor = torch.tensor(input_network).float()
emb = first_encoder(input_tensor) #Output Batch, hidden dim = 256, feature 1
emb = emb.squeeze()

In [6]:
input_tensor = torch.tensor([[1],[2],[3],[0]], dtype=torch.long)
sol =  torch.tensor([1,1,0])

In [7]:
#The two layers
layer = torch.nn.Embedding(4,2)
final_layer = nn.Linear(2,1)

In [8]:
layer(input_tensor)

tensor([[[-1.5408,  0.1991]],

        [[ 0.5197, -1.3244]],

        [[ 1.3603, -1.0763]],

        [[-0.1120, -0.2936]]], grad_fn=<EmbeddingBackward>)

Decoder architecture. Data assumptions:
<ol>
<li>Target sequence of 5 differnt tokens with length of 6. 0 token represent a padding token
<li>Batch dimensions of 2
</ol>


In [9]:
src = torch.tensor([[1,2,4,3,2,0],[1,2,3,1,1,1]],dtype=torch.long)
pos = torch.arange(0, src.shape[1]).unsqueeze(0).repeat(2, 1)
n_tokens = int(src.max()+1)
length_sequence = int(src.shape[1])
print(src)
print(pos)
print("Number of tokens output:", n_tokens)
print("Sequence Length", length_sequence)



tensor([[1, 2, 4, 3, 2, 0],
        [1, 2, 3, 1, 1, 1]])
tensor([[0, 1, 2, 3, 4, 5],
        [0, 1, 2, 3, 4, 5]])
Number of tokens output: 5
Sequence Length 6


Decoder Architecture. Parameters assumptions:
<ol>
<li> Embedding with an embedding dimension of 2
<li> Hidden Dimension twice as big as the embedding. There is an linear layer mapping the embedding to the Linear Layer
<li> 10 Convolutionals layers with kernel size of 3

In [11]:
EMBEDDING_DIM = 2

tok_embedded = nn.Embedding(n_tokens,EMBEDDING_DIM)(src)
pos_embedded = nn.Embedding(length_sequence,EMBEDDING_DIM)(pos)
print("Batch, Sequence Length, Embedding: {} \n".format(tok_embedded.shape))
print(tok_embedded)
print(pos_embedded.shape)
print(pos_embedded)


Batch, Sequence Length, Embedding: torch.Size([2, 6, 2]) 

tensor([[[-0.3273,  0.1837],
         [ 0.3341,  1.9240],
         [ 0.1831,  1.0833],
         [-1.8204,  0.0126],
         [ 0.3341,  1.9240],
         [-1.5161, -0.8355]],

        [[-0.3273,  0.1837],
         [ 0.3341,  1.9240],
         [-1.8204,  0.0126],
         [-0.3273,  0.1837],
         [-0.3273,  0.1837],
         [-0.3273,  0.1837]]], grad_fn=<EmbeddingBackward>)
torch.Size([2, 6, 2])
tensor([[[ 0.2376,  0.4152],
         [ 3.0028, -0.5745],
         [ 0.2114, -0.2161],
         [-0.2862, -0.2971],
         [-0.1592,  0.8139],
         [ 0.7925,  0.1732]],

        [[ 0.2376,  0.4152],
         [ 3.0028, -0.5745],
         [ 0.2114, -0.2161],
         [-0.2862, -0.2971],
         [-0.1592,  0.8139],
         [ 0.7925,  0.1732]]], grad_fn=<EmbeddingBackward>)


In [12]:
embedded = nn.Dropout(0.25)(tok_embedded + pos_embedded) #No influence at evaluation, but default is training
print(embedded)

tensor([[[-0.1196,  0.0000],
         [ 4.4492,  1.7994],
         [ 0.0000,  0.0000],
         [-2.8087, -0.3794],
         [ 0.0000,  3.6506],
         [-0.9648, -0.8831]],

        [[-0.1196,  0.7985],
         [ 4.4492,  0.0000],
         [-0.0000, -0.0000],
         [-0.8180, -0.1512],
         [-0.6487,  1.3301],
         [ 0.0000,  0.4758]]], grad_fn=<MulBackward0>)


In [13]:
HIDDEN_DIM = 4
hidden_output = nn.Linear(2,HIDDEN_DIM)(embedded)
print("Batch {}, Length sequence {}, Hidden state {}".format(hidden_output.shape[0],hidden_output.shape[1],hidden_output.shape[2]))
print(hidden_output)

print("\n \n Preparing for CNN \n")
conv_input = hidden_output.permute(0, 2, 1) 
print("Batch {}, Hidden sequence {}, Length state {}".format(conv_input.shape[0],conv_input.shape[1],conv_input.shape[2]))
print(conv_input)

Batch 2, Length sequence 6, Hidden state 4
tensor([[[-0.7705,  0.1613,  0.4604,  0.2774],
         [ 1.3464,  1.8496, -2.1123, -2.4623],
         [-0.7064,  0.1867,  0.3935,  0.2236],
         [-2.1415, -0.5609,  1.9675,  1.6305],
         [-1.3780,  1.6444,  0.3553, -1.1694],
         [-1.0609, -0.3707,  0.9421,  0.9942]],

        [[-0.9174,  0.4802,  0.4520, -0.0273],
         [ 1.6774,  1.1311, -2.0935, -1.7757],
         [-0.7064,  0.1867,  0.3935,  0.2236],
         [-1.1169, -0.0473,  0.8524,  0.6489],
         [-1.2987,  0.5801,  0.7422,  0.0076],
         [-0.7940,  0.3767,  0.3885,  0.0421]]], grad_fn=<AddBackward0>)

 
 Preparing for CNN 

Batch 2, Hidden sequence 4, Length state 6
tensor([[[-0.7705,  1.3464, -0.7064, -2.1415, -1.3780, -1.0609],
         [ 0.1613,  1.8496,  0.1867, -0.5609,  1.6444, -0.3707],
         [ 0.4604, -2.1123,  0.3935,  1.9675,  0.3553,  0.9421],
         [ 0.2774, -2.4623,  0.2236,  1.6305, -1.1694,  0.9942]],

        [[-0.9174,  1.6774, -0.7064,

# Convolution Time
For the seek of learning we decompose the for loop in two parts. One cycle analyzed on its own, the rest normal

In [39]:
KERNEL_SIZE = 3
PADDING_INDEX = 0
print(conv_input)
conv_input_dropped = nn.Dropout(0.25)(conv_input)
print(conv_input_dropped)

tensor([[[ 4.2386e-01,  1.4308e+00,  5.5319e-01,  2.7419e-01,  2.2194e-01,
          -5.7322e-01],
         [ 1.3442e+00,  1.0927e+00, -1.7025e+00, -2.3416e-01,  1.5453e+00,
          -1.3018e+00],
         [ 8.1805e-01,  2.6739e+00,  1.4731e+00,  7.6560e-01,  4.2511e-01,
          -6.1928e-01],
         [ 9.9371e-01, -7.9609e-01, -5.6210e-01,  5.4903e-01,  1.4189e+00,
           1.4925e+00]],

        [[ 4.5974e-01,  1.4308e+00,  5.6545e-01, -2.1756e-04,  2.3541e-02,
           3.1811e-02],
         [ 1.3131e+00,  1.0927e+00, -4.2932e-01, -3.9743e-03,  3.8912e-02,
          -6.3682e-01],
         [ 8.8725e-01,  2.6739e+00,  1.3193e+00,  2.3753e-01,  2.7456e-01,
           3.8294e-01],
         [ 9.2018e-01, -7.9609e-01, -2.2515e-02,  1.1079e+00,  1.0871e+00,
           7.7609e-01]]], grad_fn=<PermuteBackward>)
tensor([[[ 5.6514e-01,  1.9078e+00,  7.3758e-01,  3.6559e-01,  0.0000e+00,
          -7.6429e-01],
         [ 1.7923e+00,  1.4569e+00, -2.2700e+00, -3.1221e-01,  2.0603e+00,
   

In [40]:
print("Preparing Padding")
preparing_padding_cake = torch.zeros(2, HIDDEN_DIM, KERNEL_SIZE-1)
print(preparing_padding_cake)
padding = preparing_padding_cake.fill_(PADDING_INDEX) #In our case this command doesn't do much, as the padding index is zero
print(padding)

Preparing Padding
tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])
tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])


In [41]:
padded_conv_input = torch.cat((padding, conv_input_dropped), dim = 2)
print("Before Convolutions \n")
print(padded_conv_input.shape, "\n")
print(padded_conv_input)
print("After Convolution \n")
conved = nn.Conv1d(in_channels = HIDDEN_DIM, out_channels = 2 * HIDDEN_DIM,  kernel_size = KERNEL_SIZE)(padded_conv_input)
print(conved.shape, "\n")
print(conved)


Before Convolutions 

torch.Size([2, 4, 8]) 

tensor([[[ 0.0000e+00,  0.0000e+00,  5.6514e-01,  1.9078e+00,  7.3758e-01,
           3.6559e-01,  0.0000e+00, -7.6429e-01],
         [ 0.0000e+00,  0.0000e+00,  1.7923e+00,  1.4569e+00, -2.2700e+00,
          -3.1221e-01,  2.0603e+00, -0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0907e+00,  3.5651e+00,  1.9642e+00,
           1.0208e+00,  5.6682e-01, -8.2571e-01],
         [ 0.0000e+00,  0.0000e+00,  1.3250e+00, -0.0000e+00, -7.4947e-01,
           0.0000e+00,  0.0000e+00,  1.9900e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  7.5393e-01,
          -2.9008e-04,  3.1387e-02,  4.2415e-02],
         [ 0.0000e+00,  0.0000e+00,  1.7508e+00,  1.4569e+00, -0.0000e+00,
          -5.2991e-03,  5.1883e-02, -8.4909e-01],
         [ 0.0000e+00,  0.0000e+00,  1.1830e+00,  3.5651e+00,  0.0000e+00,
           3.1671e-01,  3.6609e-01,  5.1059e-01],
         [ 0.0000e+00,  0.0000e+00,  1.2269e+00, -1.0615e+00, -3.0020e-02,
  

So far everything was linear. GLU activation function is the game changer 

In [42]:
glu_conved = F.glu(conved, dim = 1)
print(glu_conved.shape)
print("\n", glu_conved)

torch.Size([2, 4, 6])

 tensor([[[ 0.1169, -0.4984, -1.5139,  0.3662,  0.1266, -0.1585],
         [ 0.0128, -0.0475, -0.7918, -0.2927,  0.4881,  0.2531],
         [-0.0339,  0.0283, -0.8370,  0.4422,  0.3688, -0.3382],
         [ 0.1113, -0.1118, -0.9893, -0.7483,  0.1450,  0.2522]],

        [[ 0.0609, -0.6341, -0.5681,  0.7505,  0.1129, -0.1424],
         [-0.0352, -0.2867, -0.2483,  0.0687, -0.2361, -0.4467],
         [-0.0920, -0.1604, -0.3074,  0.3161,  0.0379, -0.1393],
         [ 0.1185, -0.1356, -0.2852, -0.0947, -0.0337, -0.3117]]],
       grad_fn=<GluBackward>)


### Starting the Attention mechanism part

In [43]:
conved_back = glu_conved.permute(0, 2, 1)
print(conved_back)
conved_emb = nn.Linear(HIDDEN_DIM, EMBEDDING_DIM)(conved_back)
print("\n")
print(conved_emb)

tensor([[[ 0.1169,  0.0128, -0.0339,  0.1113],
         [-0.4984, -0.0475,  0.0283, -0.1118],
         [-1.5139, -0.7918, -0.8370, -0.9893],
         [ 0.3662, -0.2927,  0.4422, -0.7483],
         [ 0.1266,  0.4881,  0.3688,  0.1450],
         [-0.1585,  0.2531, -0.3382,  0.2522]],

        [[ 0.0609, -0.0352, -0.0920,  0.1185],
         [-0.6341, -0.2867, -0.1604, -0.1356],
         [-0.5681, -0.2483, -0.3074, -0.2852],
         [ 0.7505,  0.0687,  0.3161, -0.0947],
         [ 0.1129, -0.2361,  0.0379, -0.0337],
         [-0.1424, -0.4467, -0.1393, -0.3117]]], grad_fn=<PermuteBackward>)


tensor([[[-0.3505,  0.4517],
         [-0.0373,  0.1732],
         [ 1.2832, -0.3942],
         [-0.1859, -0.0703],
         [-0.7333,  0.5150],
         [-0.3043,  0.6273]],

        [[-0.2883,  0.4411],
         [ 0.2026,  0.1014],
         [ 0.2558,  0.1283],
         [-0.6725,  0.4418],
         [-0.2060,  0.2789],
         [ 0.1419,  0.0931]]], grad_fn=<AddBackward0>)


In [44]:
 SCALE = torch.sqrt(torch.FloatTensor([0.5]))
 combined = (conved_emb + embedded) * SCALE
 print(combined)

tensor([[[-0.2479, -1.4382],
         [-2.3309, -2.2747],
         [-0.3814,  0.9833],
         [-0.3211, -0.0497],
         [-0.0068, -1.4203],
         [ 1.1133,  2.2982]],

        [[-0.2933, -1.4457],
         [-2.1613, -2.3255],
         [-0.7156,  0.0317],
         [ 0.0161,  0.3204],
         [ 0.3077,  0.1398],
         [ 0.3129,  0.6964]]], grad_fn=<MulBackward0>)


## Attention Mechanism 

In [48]:
energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))

NameError: name 'encoder_conved' is not defined

In [49]:
a = torch.ones((2,2))
b = torch.ones((2,2))
torch.matmul(a,b)

tensor([[2., 2.],
        [2., 2.]])

In [50]:
torch.ones((2,2))

tensor([[1., 1.],
        [1., 1.]])

In [51]:
CONV_LAYERS = 10
for i, conv in enumerate(CONV_LAYERS-1):
    conv_input = nn.Dropout(0.25)(conv_input)


TypeError: 'int' object is not iterable

def calculate_attention(self, embedded, conved, encoder_conved, encoder_combined):
    
    #embedded = [batch size, trg len, emb dim]
    #conved = [batch size, hid dim, trg len]
    #encoder_conved = encoder_combined = [batch size, src len, emb dim]
    
    #permute and convert back to emb dim
    conved_emb = self.attn_hid2emb(conved.permute(0, 2, 1))
    
    #conved_emb = [batch size, trg len, emb dim]
    
    combined = (conved_emb + embedded) * self.scale
    
    #combined = [batch size, trg len, emb dim]
            
    energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))
    
    #energy = [batch size, trg len, src len]
    
    attention = F.softmax(energy, dim=2)
    
    #attention = [batch size, trg len, src len]
        
    attended_encoding = torch.matmul(attention, encoder_combined)
    
    #attended_encoding = [batch size, trg len, emd dim]
    
    #convert from emb dim -> hid dim
    attended_encoding = self.attn_emb2hid(attended_encoding)
    
    #attended_encoding = [batch size, trg len, hid dim]
    
    #apply residual connection
    attended_combined = (conved + attended_encoding.permute(0, 2, 1)) * self.scale
    
    #attended_combined = [batch size, hid dim, trg len]
    
    return attention, attended_combined